In [89]:
import os
import numpy as np
import pandas as pd
import tempfile

import pandapower as pp
import pandapower.networks as nw
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import pandapower.plotting as pplt
import matplotlib.pyplot as plt
from pandapower.plotting.plotly import pf_res_plotly

#creating our net
net = nw.create_cigre_network_mv()

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')

#print(net)

#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, color ="g", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()

#Adding CS at rated power (6.6 kW)


max_line_loading=100

max_vm_pu= 1.1

min_vm_pu= 0.9



In [90]:

#importing the active and reactive power of loads from excel
df_active = pd.read_csv('active_power_time_series_ev_load.csv')
df_reactive = pd.read_csv('reactive_power_time_series_ev_load.csv')
# create the data source from
ds_active = DFData(df_active)
ds_reactive = DFData(df_reactive)

def timeseries_example(output_dir):
    
    #defing the time step
    n_timesteps = 24
    
    # 3. create controllers (to control P values of the load) ds_p is for active power and ds_q for reactive power
    create_controllers(net, ds_active, ds_reactive)

    # time steps to be calculated. Could also be a list with non-consecutive time steps
    time_steps = range(0, n_timesteps)

    # 4. the output writer with the desired results to be stored to files.
    ow = create_output_writer(net, time_steps, output_dir=output_dir)

    # 5. the main time series function
    run_timeseries(net, time_steps) 
    
    
load_index=net.load.index
load_name=net.load.name
def create_controllers(net,ds_active,ds_reactive):
    for i in range(len(load_index)):
        ConstControl(net, element='load', variable='p_mw', element_index=load_index[i],
                 data_source=ds_active, profile_name=load_name[i])
        ConstControl(net, element='load', variable='q_mvar', element_index=load_index[i],
                 data_source=ds_reactive, profile_name=load_name[i])

def create_output_writer(net, time_steps, output_dir):
    ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())
    # these variables are saved to the harddisk after / during the time series loop
    ow.log_variable('res_load', 'p_mw')
    ow.log_variable('res_bus', 'vm_pu')
    ow.log_variable('res_line', 'loading_percent')
    ow.log_variable('res_line', 'i_ka')
    ow.log_variable('res_trafo','loading_percent')
    return ow




In [91]:
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

Results can be found in your local temp folder: time_series_ev



100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 128.42it/s]


In [92]:
#Bus CS 14 (0.63 MVA) - trafo id: 2

#for i in range (1,41):
 #   l = str(i)
  #  pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

#Bus CS 11 (0.4 MVA) - trafo id: 3
#for i in range (41,60):
 #   l = str(i)
  #  pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

#Bus CS 5 (0.4 MVA) - trafo id: 4
#for i in range (61,80):
 #   l = str(i)
  #  pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
#Bus CS 4 (0.25 MVA) - trafo id: 5
#for i in range (81,90):
 #   l = str(i)
  #  pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
#Bus CS 9 (0.25 MVA)- trafo id: 6
#for i in range (91,101):
 #   l = str(i)
  #  pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)




In [93]:
ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

if max(line_loading)>100:
    print("Overloaded lines")



vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)
a=vm_pu

if max(vm_pu)>max_vm_pu:
    print("voltage over maximum")
    
elif min(vm_pu)<min_vm_pu:
        
     print("voltage below minimum")
        
    


voltage over maximum


In [94]:
#Bus CS 14 (0.63 MVA) - trafo id: 2

for i in range (1,400):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=i
while all(vm_pu.max())<max_vm_pu and all(line_loading.max())<100 and all(vm_pu.min())>min_vm_pu and c<450:
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)

    if any(line_loading.max())>100:
        print("Overloaded lines")

    elif any(vm_pu.max())>max_vm_pu:
        print("voltage over maximum")

    elif any(vm_pu.min())<min_vm_pu:
         print("voltage below minimum")
            
    elif c==450:
        print("no maximum has been found")



400
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 85.74it/s]


401
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 78.58it/s]


402
Results can be found in your local temp folder: time_series_ev


...

449
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.98it/s]


450
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.81it/s]

no maximum has been found


In [95]:
line_loading

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,24.439443,24.734911,9.696833,7.238953,3.136339,0.566307,8.584885,5.021206,1.891682,12.234007,54.520528,54.373469,0.091468,0.186601,0.036524
1,7.513364,7.038936,2.641008,1.978996,0.951739,0.589315,2.220915,1.354775,0.555322,3.271636,51.862337,51.835998,0.093257,0.190375,0.036832
2,5.611273,4.580948,1.213176,0.947155,0.676175,0.634533,0.793972,0.641413,0.327573,1.815974,51.146836,51.153439,0.093731,0.191374,0.036914
3,5.768426,4.673707,1.131259,0.896509,0.684405,0.648344,0.712270,0.608317,0.318254,1.846094,50.961089,50.976274,0.093853,0.191632,0.036935
4,5.857754,4.757885,1.146328,0.909141,0.692907,0.652108,0.736312,0.617806,0.320733,1.891193,50.912175,50.929621,0.093885,0.191700,0.036940
5,5.883260,4.782807,1.152528,0.914012,0.695452,0.653066,0.745236,0.621374,0.321680,1.904819,50.899832,50.917850,0.093894,0.191717,0.036942
6,5.889953,4.789394,1.154261,0.915360,0.696127,0.653311,0.747685,0.622356,0.321942,1.908433,50.896684,50.914847,0.093896,0.191722,0.036942
7,5.891702,4.791119,1.154721,0.915716,0.696303,0.653374,0.748332,0.622616,0.322011,1.909381,50.895866,50.914067,0.093896,0.191723,0.036942
8,5.892145,4.791556,1.154838,0.915807,0.696347,0.653391,0.748496,0.622682,0.322029,1.909620,50.895659,50.913870,0.093896,0.191723,0.036942
9,5.892300,4.791709,1.154879,0.915839,0.696363,0.653396,0.748554,0.622706,0.322035,1.909705,50.895587,50.913801,0.093896,0.191723,0.036942


In [96]:
vm_pu

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.03,1.022289,1.017222,1.008787,1.008343,1.008041,1.007691,1.007450,1.007487,1.007246,1.006946,1.006900,1.016075,0.988832,0.972242,0.848790,1.006876,1.008017,1.008320,1.007223
1,1.03,1.029070,1.028707,1.027587,1.027508,1.027457,1.027403,1.027405,1.027379,1.027328,1.027271,1.027264,1.022052,0.996222,0.980457,0.859837,1.027239,1.027432,1.027485,1.027305
2,1.03,1.030816,1.031711,1.032557,1.032577,1.032593,1.032619,1.032686,1.032643,1.032644,1.032652,1.032656,1.023618,0.998169,0.982625,0.862728,1.032631,1.032568,1.032553,1.032620
3,1.03,1.031263,1.032485,1.033840,1.033886,1.033920,1.033967,1.034051,1.034003,1.034017,1.034043,1.034049,1.024022,0.998671,0.983185,0.863473,1.034024,1.033895,1.033863,1.033994
4,1.03,1.031380,1.032688,1.034178,1.034231,1.034269,1.034322,1.034410,1.034361,1.034379,1.034408,1.034415,1.024128,0.998803,0.983332,0.863668,1.034390,1.034244,1.034207,1.034355
5,1.03,1.031410,1.032740,1.034263,1.034318,1.034357,1.034411,1.034500,1.034451,1.034470,1.034501,1.034508,1.024154,0.998836,0.983369,0.863718,1.034483,1.034332,1.034294,1.034446
6,1.03,1.031417,1.032753,1.034285,1.034340,1.034379,1.034434,1.034523,1.034474,1.034493,1.034524,1.034531,1.024161,0.998845,0.983378,0.863730,1.034506,1.034355,1.034316,1.034469
7,1.03,1.031419,1.032756,1.034291,1.034346,1.034385,1.034440,1.034529,1.034480,1.034499,1.034530,1.034537,1.024163,0.998847,0.983381,0.863734,1.034513,1.034360,1.034322,1.034475
8,1.03,1.031420,1.032757,1.034292,1.034347,1.034387,1.034441,1.034531,1.034482,1.034501,1.034532,1.034539,1.024164,0.998847,0.983382,0.863734,1.034514,1.034362,1.034323,1.034477
9,1.03,1.031420,1.032757,1.034293,1.034348,1.034387,1.034442,1.034531,1.034482,1.034501,1.034532,1.034539,1.024164,0.998848,0.983382,0.863735,1.034515,1.034362,1.034324,1.034477


In [97]:
vm_pu.min()

0     1.030000
1     1.022289
2     1.017222
3     1.008787
4     1.008343
5     1.008041
6     1.007691
7     1.007450
8     1.007487
9     1.007246
10    1.006946
11    1.006900
12    1.016075
13    0.988832
14    0.972242
15    0.848790
16    1.006876
17    1.008017
18    1.008320
19    1.007223
dtype: float64

In [98]:
any(vm_pu.min())<2

True

In [99]:
max(line_loading)

14

In [100]:
all(vm_pu.min()<min_vm_pu)

False

In [101]:
all(vm_pu.max())<max_vm_pu and all(line_loading.max())<100 and all(vm_pu.min())>min_vm_pu and c<450

False